# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
#importanto bibliotecas
import pandas as pd
import seaborn as sns
from seaborn import load_dataset
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
df = pd.read_csv('/kaggle/input/previsao-de-renda/previsao_de_renda.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   index                  15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12466 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  mau                    15000 non-null  bool   
 15  re

In [4]:
#dropando os dados missing
df = df.dropna()

In [10]:
#dropando colunas indesejadas
df = df.drop(['Unnamed: 0','data_ref','index'], axis = 1)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12466 entries, 0 to 14999
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   sexo                   12466 non-null  object 
 1   posse_de_veiculo       12466 non-null  bool   
 2   posse_de_imovel        12466 non-null  bool   
 3   qtd_filhos             12466 non-null  int64  
 4   tipo_renda             12466 non-null  object 
 5   educacao               12466 non-null  object 
 6   estado_civil           12466 non-null  object 
 7   tipo_residencia        12466 non-null  object 
 8   idade                  12466 non-null  int64  
 9   tempo_emprego          12466 non-null  float64
 10  qt_pessoas_residencia  12466 non-null  float64
 11  mau                    12466 non-null  bool   
 12  renda                  12466 non-null  float64
dtypes: bool(3), float64(3), int64(2), object(5)
memory usage: 1.3+ MB


In [30]:
df.sexo.value_counts()

sexo
F    7991
M    4475
Name: count, dtype: int64

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [26]:
#ajustando o modelo 1 log(renda) pelas variaveis disponiveis.

y, X = patsy.dmatrices('np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + tipo_renda + educacao + estado_civil + tipo_residencia + idade + tempo_emprego + qt_pessoas_residencia + 0', df)
X

DesignMatrix with shape (12466, 25)
  Columns:
    ['sexo[F]',
     'sexo[M]',
     'posse_de_veiculo[T.True]',
     'posse_de_imovel[T.True]',
     'tipo_renda[T.Bolsista]',
     'tipo_renda[T.Empresário]',
     'tipo_renda[T.Pensionista]',
     'tipo_renda[T.Servidor público]',
     'educacao[T.Pós graduação]',
     'educacao[T.Secundário]',
     'educacao[T.Superior completo]',
     'educacao[T.Superior incompleto]',
     'estado_civil[T.Separado]',
     'estado_civil[T.Solteiro]',
     'estado_civil[T.União]',
     'estado_civil[T.Viúvo]',
     'tipo_residencia[T.Casa]',
     'tipo_residencia[T.Com os pais]',
     'tipo_residencia[T.Comunitário]',
     'tipo_residencia[T.Estúdio]',
     'tipo_residencia[T.Governamental]',
     'qtd_filhos',
     'idade',
     'tempo_emprego',
     'qt_pessoas_residencia']
  Terms:
    'sexo' (columns 0:2)
    'posse_de_veiculo' (column 2)
    'posse_de_imovel' (column 3)
    'tipo_renda' (columns 4:8)
    'educacao' (columns 8:12)
    'estado_civil

In [27]:
#MODELO 1
modelo1 = sm.OLS(y,X).fit()

In [28]:
# Verificando a sg=ignificancia das variaveis
modelo1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.242
Model:                            OLS   Adj. R-squared:                  0.240
Method:                 Least Squares   F-statistic:                     165.3
Date:                Mon, 27 May 2024   Prob (F-statistic):               0.00
Time:                        14:24:14   Log-Likelihood:                -13204.
No. Observations:               12466   AIC:                         2.646e+04
Df Residuals:                   12441   BIC:                         2.664e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
sexo[F]                              7.2080      0.272     26.527      0.000       6.675       7.741
sexo[M]                              7.6893      0.271     28.333      0.000       7.157       8.221
posse_de_veiculo[T.True]            -0.0012      0.014     -0.087      0.931      -0.028       0.026
posse_de_imovel[T.True]              0.0849      0.014      6.275      0.000       0.058       0.111
tipo_renda[T.Bolsista]              -0.0493      0.350     -0.141      0.888      -0.735       0.636
tipo_renda[T.Empresário]             0.1786      0.015     12.209      0.000       0.150       0.207
tipo_renda[T.Pensionista]            0.3784      0.233      1.623      0.105      -0.079       0.835
tipo_renda[T.Servidor público]       0.0950      0.021      4.485      0.000       0.053       0.136
educacao[T.Pós graduação]            0.5502      0.175      3.137      0.002       0.206       0.894
educacao[T.Secundário]               0.0164      0.071      0.232      0.817      -0.123       0.156
educacao[T.Superior completo]        0.1132      0.071      1.587      0.112      -0.027       0.253
educacao[T.Superior incompleto]     -0.0097      0.076     -0.127      0.899      -0.159       0.140
estado_civil[T.Separado]            -0.0437      0.131     -0.334      0.738      -0.300       0.213
estado_civil[T.Solteiro]             0.0169      0.128      0.131      0.895      -0.235       0.269
estado_civil[T.União]               -0.0259      0.023     -1.110      0.267      -0.072       0.020
estado_civil[T.Viúvo]               -0.0422      0.134     -0.314      0.753      -0.306       0.221
tipo_residencia[T.Casa]              0.0148      0.053      0.278      0.781      -0.090       0.119
tipo_residencia[T.Com os pais]      -0.0572      0.060     -0.957      0.339      -0.174       0.060
tipo_residencia[T.Comunitário]       0.0575      0.103      0.559      0.576      -0.144       0.259
tipo_residencia[T.Estúdio]          -0.1094      0.089     -1.232      0.218      -0.283       0.065
tipo_residencia[T.Governamental]    -0.0563      0.064     -0.873      0.382      -0.183       0.070
qtd_filhos                           0.0087      0.128      0.068      0.946      -0.242       0.260
idade                                0.0052      0.001      6.625      0.000       0.004       0.007
tempo_emprego                        0.0482      0.001     48.426      0.000       0.046       0.050
qt_pessoas_residencia                0.0073      0.128      0.057      0.955      -0.243       0.258
==============================================================================
Omnibus:                        1.155   Durbin-Watson:                   2.015
Prob(Omnibus):                  0.561   Jarque-Bera (JB):                1.126
Skew:             

Analisando a tabela, as variavies 'estado_civil', 'tipo_residencia', 'qtd_filhos', 'qt_pessoas_residencia', 'posse_de_veiculo' pelo P.Value não se mostraram significantes então serão removidas.

In [31]:
y, X = patsy.dmatrices('np.log(renda) ~ sexo + posse_de_imovel + tipo_renda + educacao + idade + tempo_emprego + 0', df)
X

DesignMatrix with shape (12466, 13)
  Columns:
    ['sexo[F]',
     'sexo[M]',
     'posse_de_imovel[T.True]',
     'tipo_renda[T.Bolsista]',
     'tipo_renda[T.Empresário]',
     'tipo_renda[T.Pensionista]',
     'tipo_renda[T.Servidor público]',
     'educacao[T.Pós graduação]',
     'educacao[T.Secundário]',
     'educacao[T.Superior completo]',
     'educacao[T.Superior incompleto]',
     'idade',
     'tempo_emprego']
  Terms:
    'sexo' (columns 0:2)
    'posse_de_imovel' (column 2)
    'tipo_renda' (columns 3:7)
    'educacao' (columns 7:11)
    'idade' (column 11)
    'tempo_emprego' (column 12)
  (to view full data, use np.asarray(this_obj))

In [33]:
#MODELO 2
modelo2 = sm.OLS(y,X).fit()

In [34]:
modelo2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.240
Model:                            OLS   Adj. R-squared:                  0.240
Method:                 Least Squares   F-statistic:                     328.5
Date:                Mon, 27 May 2024   Prob (F-statistic):               0.00
Time:                        14:30:59   Log-Likelihood:                -13215.
No. Observations:               12466   AIC:                         2.646e+04
Df Residuals:                   12453   BIC:                         2.655e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
sexo[F]                             7.2288      0.076     95.409      0.000       7.080       7.377
sexo[M]                             7.7140      0.075    102.744      0.000       7.567       7.861
posse_de_imovel[T.True]             0.0959      0.013      7.258      0.000       0.070       0.122
tipo_renda[T.Bolsista]             -0.0446      0.350     -0.127      0.899      -0.730       0.641
tipo_renda[T.Empresário]            0.1778      0.015     12.186      0.000       0.149       0.206
tipo_renda[T.Pensionista]           0.3880      0.233      1.664      0.096      -0.069       0.845
tipo_renda[T.Servidor público]      0.0941      0.021      4.455      0.000       0.053       0.135
educacao[T.Pós graduação]           0.5752      0.175      3.283      0.001       0.232       0.919
educacao[T.Secundário]              0.0274      0.071      0.387      0.699      -0.111       0.166
educacao[T.Superior completo]       0.1243      0.071      1.747      0.081      -0.015       0.264
educacao[T.Superior incompleto]     0.0009      0.076      0.012      0.990      -0.148       0.150
idade                               0.0048      0.001      6.602      0.000       0.003       0.006
tempo_emprego                       0.0484      0.001     48.683      0.000       0.046       0.050
==============================================================================
Omnibus:                        0.894   Durbin-Watson:                   2.014
Prob(Omnibus):                  0.640   Jarque-Bera (JB):                0.861
Skew:                          -0.012   Prob(JB):                        0.650
Kurtosis:                       3.033   Cond. No.                     2.38e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.38e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Comparando os dois modelos ate o momento não sofreu muita diferença na pratica após a remoção das variaveis menos significantes. Mas pelo fato de apresentarem um resultado semelhante vamos ficar com o modelo mais simples. No caso o modelo 2. Agora vou remover a variavel 'educacao', uma vez que se mostra em sua maioria insignificante. Mesmo com as pessoas com 'pos-graduaçao' tem uma certa relevancia levando em conta a porcentagem dessas pessoas na base é muito baixa, será removido tambem.

In [35]:
y, X = patsy.dmatrices('np.log(renda) ~ sexo + posse_de_imovel + tipo_renda + idade + tempo_emprego + 0', df)
X

DesignMatrix with shape (12466, 9)
  Columns:
    ['sexo[F]',
     'sexo[M]',
     'posse_de_imovel[T.True]',
     'tipo_renda[T.Bolsista]',
     'tipo_renda[T.Empresário]',
     'tipo_renda[T.Pensionista]',
     'tipo_renda[T.Servidor público]',
     'idade',
     'tempo_emprego']
  Terms:
    'sexo' (columns 0:2)
    'posse_de_imovel' (column 2)
    'tipo_renda' (columns 3:7)
    'idade' (column 7)
    'tempo_emprego' (column 8)
  (to view full data, use np.asarray(this_obj))

In [36]:
#MODELO 3

modelo3 = sm.OLS(y,X).fit()

In [37]:
modelo3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.236
Model:                            OLS   Adj. R-squared:                  0.236
Method:                 Least Squares   F-statistic:                     481.8
Date:                Mon, 27 May 2024   Prob (F-statistic):               0.00
Time:                        14:36:47   Log-Likelihood:                -13249.
No. Observations:               12466   AIC:                         2.652e+04
Df Residuals:                   12457   BIC:                         2.658e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
sexo[F]                            7.2988      0.031    238.636      0.000       7.239       7.359
sexo[M]                            7.7807      0.030    263.062      0.000       7.723       7.839
posse_de_imovel[T.True]            0.1009      0.013      7.625      0.000       0.075       0.127
tipo_renda[T.Bolsista]             0.0213      0.351      0.061      0.951      -0.666       0.708
tipo_renda[T.Empresário]           0.1866      0.015     12.806      0.000       0.158       0.215
tipo_renda[T.Pensionista]          0.4337      0.234      1.856      0.063      -0.024       0.892
tipo_renda[T.Servidor público]     0.1082      0.021      5.133      0.000       0.067       0.149
idade                              0.0045      0.001      6.239      0.000       0.003       0.006
tempo_emprego                      0.0484      0.001     48.558      0.000       0.046       0.050
==============================================================================
Omnibus:                        0.874   Durbin-Watson:                   2.011
Prob(Omnibus):                  0.646   Jarque-Bera (JB):                0.845
Skew:                          -0.004   Prob(JB):                        0.656
Kurtosis:                       3.039   Cond. No.                     2.38e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.38e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [42]:
y, X = patsy.dmatrices('np.log(renda) ~ sexo + posse_de_imovel + idade + tempo_emprego', df)
X

DesignMatrix with shape (12466, 5)
  Intercept  sexo[T.M]  posse_de_imovel[T.True]  idade  tempo_emprego
          1          0                        1     36        3.57534
          1          1                        1     42        0.86027
          1          1                        1     31        8.06575
          1          0                        0     50        1.20822
          1          1                        0     52       13.87397
          1          1                        1     46       15.56164
          1          0                        1     51        5.04658
          1          1                        1     24        1.59178
          1          0                        0     53        3.43014
          1          0                        1     34        7.21918
          1          0                        1     35        5.63836
          1          0                        1     31        4.47945
          1          0                        1     50 

In [43]:
#MODELO 4

modelo4 = sm.OLS(y,X).fit()

In [44]:
modelo4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.226
Model:                            OLS   Adj. R-squared:                  0.225
Method:                 Least Squares   F-statistic:                     907.7
Date:                Mon, 27 May 2024   Prob (F-statistic):               0.00
Time:                        14:40:24   Log-Likelihood:                -13335.
No. Observations:               12466   AIC:                         2.668e+04
Df Residuals:                   12461   BIC:                         2.672e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                   7.3732      0.030    243.856      0.000       7.314       7.432
sexo[T.M]                   0.4742      0.013     35.424      0.000       0.448       0.500
posse_de_imovel[T.True]     0.1018      0.013      7.649      0.000       0.076       0.128
idade                       0.0043      0.001      5.958      0.000       0.003       0.006
tempo_emprego               0.0479      0.001     48.283      0.000       0.046       0.050
==============================================================================
Omnibus:                        1.296   Durbin-Watson:                   2.008
Prob(Omnibus):                  0.523   Jarque-Bera (JB):                1.272
Skew:                           0.011   Prob(JB):                        0.529
Kurtosis:                       3.045   Cond. No.                         207.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Foi realizado a análise de 4 modelos. Apesar do modelo final possuir apenas variaveis significantes olhando para o P.Value, temos uma 'piora' no modelo em relacao aos modelos anteriores. Comparando com o melhor modelo na minha opniao que é o modelo 2 temos:

* O Modelo 2 possui um R-Quadrado (.240) melhor que o Modelo 4 (.226) (modelo final)  
* O Modelo 2 possui um R-Quadrado ajustado (.240) melhor que o modelo 4 (.225)
* O Modelo 2 possui um índice de AIC (2.646e+04) mais baixo que o modelo 4 (2.668e+04), logo um indice melhor

Sendo assim o melhor modelo avaliando apenas as informações que o sumário oferece, é o modelo 2, por se aprensar mais simples que o modelo 1 e nao perder 'qualidade'.